In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tensorflow.keras.layers import Dropout
from concurrent.futures import ProcessPoolExecutor
import tensorflow as tf
from tensorflow import keras
import datetime,time
import os
import random
from datetime import date

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#data_online_path =r'/content/drive//MyDrive/Tmall_O2O/ccf_online_stage1_train.csv')
#data_offline_path=r'./content/drive//MyDrive/Tmall_O2O/ccf_offline_stage1_train.csv')

In [ ]:
def get_source_data():
    DataPath = '/content/drive//MyDrive/Tmall_O2O/'
    off_train = pd.read_csv(os.path.join(DataPath, 'ccf_offline_stage1_train.csv'),
                            parse_dates=['Date_received', 'Date'])
    off_train.columns = ['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance', 'Date_received', 'Date']

    on_train = pd.read_csv(os.path.join(DataPath, 'ccf_online_stage1_train.csv'), parse_dates=['Date_received', 'Date'])
    on_train.columns = ['User_id', 'Merchant_id', 'Action', 'Coupon_id', 'Discount_rate', 'Date_received', 'Date']

    off_test = pd.read_csv(os.path.join(DataPath, 'ccf_offline_stage1_test_revised.csv'), parse_dates=['Date_received'])
    off_test.columns = ['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance', 'Date_received']

    print(off_train.info())
    print(off_train.head(5))
    return off_train, on_train, off_test



In [ ]:
def null_process_offline(dataset, predict=False):
    dataset.Distance.fillna(11, inplace=True)
    dataset.Distance = dataset.Distance.astype(int)
    dataset.Coupon_id.fillna(0, inplace=True)
    dataset.Coupon_id = dataset.Coupon_id.astype(int)
    dataset.Date_received.fillna(date_null, inplace=True)

    dataset[['discount_rate_x', 'discount_rate_y']] = dataset[dataset.Discount_rate.str.contains(':') == True][
        'Discount_rate'].str.split(':', expand=True).astype(int)
    dataset['discount_rate'] = 1 - dataset.discount_rate_y / dataset.discount_rate_x
    dataset.discount_rate = dataset.discount_rate.fillna(dataset.Discount_rate).astype(float)
    if predict:
        return dataset
    else:
        dataset.Date.fillna(date_null, inplace=True)
        return dataset


def null_process_online(dataset):
    dataset.Coupon_id.fillna(0, inplace=True)
    dataset.Date_received.fillna(date_null, inplace=True)
    dataset.Date.fillna(date_null, inplace=True)
    return dataset


In [ ]:
def data_process(off_train, on_train, off_test):
    time_range = ['2016-04-16', '2016-05-15']
    dataset1 = off_train[(off_train.Date_received >= time_range[0]) & (off_train.Date_received <= time_range[1])].copy()
    dataset1['label'] = 0
    dataset1.loc[
        (dataset1.Date != date_null) & (dataset1.Date - dataset1.Date_received <= datetime.timedelta(15)), 'label'] = 1
    time_range_date_received = ['2016-01-01', '2016-03-31']
    time_range_date = ['2016-01-01', '2016-04-15']
    feature1_off = off_train[(off_train.Date >= time_range_date[0]) & (off_train.Date <= time_range_date[1]) | (
            (off_train.Coupon_id == 0) & (off_train.Date_received >= time_range_date_received[0]) & (
            off_train.Date_received <= time_range_date_received[1]))]
    feature1_on = on_train[(on_train.Date >= time_range_date[0]) & (on_train.Date <= time_range_date[1]) | (
            (on_train.Coupon_id == 0) & (on_train.Date_received >= time_range_date_received[0]) & (
            on_train.Date_received <= time_range_date_received[1]))]
    time_range = ['2016-05-16', '2016-06-15']
    dataset2 = off_train[(off_train.Date_received >= time_range[0]) & (off_train.Date_received <= time_range[1])]
    dataset2['label'] = 0
    dataset2.loc[
        (dataset2.Date != date_null) & (dataset2.Date - dataset2.Date_received <= datetime.timedelta(15)), 'label'] = 1
   
    time_range_date_received = ['2016-02-01', '2016-04-30']
    time_range_date = ['2016-02-01', '2016-05-15']
    feature2_off = off_train[(off_train.Date >= time_range_date[0]) & (off_train.Date <= time_range_date[1]) | (
            (off_train.Coupon_id == 0) & (off_train.Date_received >= time_range_date_received[0]) & (
            off_train.Date_received <= time_range_date_received[1]))]
  
    feature2_on = on_train[(on_train.Date >= time_range_date[0]) & (on_train.Date <= time_range_date[1]) | (
            (on_train.Coupon_id == 0) & (on_train.Date_received >= time_range_date_received[0]) & (
            on_train.Date_received <= time_range_date_received[1]))]

   
    dataset3 = off_test
    
    time_range = ['2016-03-16', '2016-06-30']
    feature3_off = off_train[((off_train.Date >= time_range[0]) & (off_train.Date <= time_range[1])) | (
            (off_train.Coupon_id == 0) & (off_train.Date_received >= time_range[0]) & (
            off_train.Date_received <= time_range[1]))]
 
    feature3_on = on_train[((on_train.Date >= time_range[0]) & (on_train.Date <= time_range[1])) | (
            (on_train.Coupon_id == 0) & (on_train.Date_received >= time_range[0]) & (
            on_train.Date_received <= time_range[1]))]

    
    ProcessDataSet1 = get_features(dataset1, feature1_off, feature1_on)
    ProcessDataSet2 = get_features(dataset2, feature2_off, feature2_on)
    ProcessDataSet3 = get_features(dataset3, feature3_off, feature3_on)

    return ProcessDataSet1, ProcessDataSet2, ProcessDataSet3

In [ ]:
def get_features(dataset, feature_off, feature_on):
    dataset = get_offline_features(dataset, feature_off)
    return get_online_features(feature_on, dataset)

In [ ]:
def get_offline_features(X, offline):
   
    print(len(X), len(X.columns))

    temp = offline[offline.Coupon_id != 0] 
    coupon_consume = temp[temp.Date != date_null] 
    coupon_no_consume = temp[temp.Date == date_null] 

    user_coupon_consume = coupon_consume.groupby('User_id')

    X['weekday'] = X.Date_received.dt.weekday
    X['day'] = X.Date_received.dt.day

    '''user features'''

    
    temp = user_coupon_consume.size().reset_index(name='offline_user_coupon_consume')
    X = pd.merge(X, temp, how='left', on='User_id')
   

    temp = coupon_no_consume.groupby('User_id').size().reset_index(name='offline_user_coupon_no_consume')
    X = pd.merge(X, temp, how='left', on='User_id')

    X['offline_user_coupon_receive'] = X.offline_user_coupon_consume.fillna(0) + X.offline_user_coupon_no_consume.fillna(0)

   
    X['offline_user_coupon_consume'] = X.offline_user_coupon_consume / X.offline_user_coupon_receive

    temp = offline[(offline.Coupon_id == 0) & (offline.Date != date_null)]
    temp1 = temp.groupby('User_id').size().reset_index(name='offline_nocoupon_consume')
    X = pd.merge(X, temp1, how='left', on='User_id')

  
    X['total_consumptions'] = X.offline_user_coupon_consume + X.offline_nocoupon_consume

    X['%coupon_used_in_totalconsumption'] = X.offline_user_coupon_consume / X.total_consumptions

    temp = pd.merge(temp, temp.groupby('User_id').Date.max().reset_index(name='max'))
    temp = pd.merge(temp, temp.groupby('User_id').Date.min().reset_index(name='min'))
    temp = pd.merge(temp, temp.groupby('User_id').size().reset_index(name='len'))
    temp['offline_consumed_daygap'] = ((temp['max'] - temp['min']).dt.days / (temp['len'] - 1))
    temp = temp.drop_duplicates('User_id')
    X = pd.merge(X, temp[['User_id', 'offline_consumed_daygap']], how='left', on='User_id')

    temp = coupon_consume.copy()
    temp['days'] = (temp.Date - temp.Date_received).dt.days
    temp = (temp.groupby('User_id').days.sum() / temp.groupby('User_id').size()).reset_index(name='offline_days_from_received_to_consumed')
    X = pd.merge(X, temp, how='left', on='User_id')

    temp = user_coupon_consume.discount_rate.mean().reset_index(name='offline_consumed_avg_discountrate')
    X = pd.merge(X, temp, how='left', on='User_id')

    temp = offline[(offline.Coupon_id != 0) & (offline.Date != date_null) & (offline.Distance != 11)]
    temp = temp.groupby('User_id').Distance
    temp = pd.merge(temp.count().reset_index(name='x'), temp.sum().reset_index(name='y'), on='User_id')
    temp['avg_distance_consumed'] = temp.y / temp.x
    temp = temp[['User_id', 'avg_distance_consumed']]
    X = pd.merge(X, temp, how='left', on='User_id')

    temp = coupon_consume[coupon_consume.Distance != 11]
    temp = temp.groupby('User_id').Distance.min().reset_index(name='min_distance_consumed')
    X = pd.merge(X, temp, how='left', on='User_id')

    temp = coupon_consume[coupon_consume.Distance != 11]
    temp = temp.groupby('User_id').Distance.max().reset_index(name='max_distance_consumed')
    X = pd.merge(X, temp, how='left', on='User_id')

    discount_types = [
        '0.2', '0.5', '0.6', '0.7', '0.75', '0.8', '0.85', '0.9', '0.95', '30:20', '50:30', '10:5',
        '20:10', '100:50', '200:100', '50:20', '30:10', '150:50', '100:30', '20:5', '200:50', '5:1',
        '50:10', '100:20', '150:30', '30:5', '300:50', '200:30', '150:20', '10:1', '50:5', '100:10',
        '200:20', '300:30', '150:10', '300:20', '500:30', '20:1', '100:5', '200:10', '30:1', '150:5',
        '300:10', '200:5', '50:1', '100:1',
    ]
    X['discount_type'] = -1
    for k, v in enumerate(discount_types):
        X.loc[X.Discount_rate == v, 'discount_type'] = k

  
    temp = offline.groupby(['User_id', 'Discount_rate']).size().reset_index(name='offline_coupon_received_bytype')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Discount_rate'])

   
    temp = coupon_consume.groupby(['User_id', 'Discount_rate']).size().reset_index(name='offline_coupon_consumed_bytype')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Discount_rate'])


    X['%ofcouponused_bytype'] = X.offline_coupon_consumed_bytype / X.offline_coupon_received_bytype


    '''offline merchant features'''

   
    temp = offline[offline.Date != date_null].groupby('Merchant_id').size().reset_index(name='offline_merchant_no.oforder')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

   
    temp = coupon_consume.groupby('Merchant_id').size().reset_index(name='offline_merchant_coupon_recieved_consumed')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

 
    temp = offline[offline.Date_received != date_null].groupby('Merchant_id').size().reset_index(name='offline_merchant_coupon_recieved')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    X['offline_merchant_%coupon_used'] = X.offline_merchant_coupon_recieved_consumed / X.offline_merchant_coupon_recieved

  
    temp = coupon_consume.groupby('Merchant_id').discount_rate.mean().reset_index(name='offline_merchant_avgdiscountrate')
    X = pd.merge(X, temp, how='left', on='Merchant_id')


    print(len(X), len(X.columns))

    return X





In [ ]:
def get_online_features(online, X):
    
    
    temp = online.groupby('User_id').size().reset_index(name='online_user_actions')
    X = pd.merge(X, temp, how='left', on='User_id')

  
    temp = online[online.Action == 0].groupby('User_id').size().reset_index(name='online_user_clicks')
    X = pd.merge(X, temp, how='left', on='User_id')

   
    X['online_CTR'] = X.online_user_clicks / X.online_user_actions

   
    temp = online[online.Action == 1].groupby('User_id').size().reset_index(name='online_user_purchase')
    X = pd.merge(X, temp, how='left', on='User_id')

   
    X['online_PurchaseRate'] = X.online_user_purchase / X.online_user_actions

   
    temp = online[online.Coupon_id != 0].groupby('User_id').size().reset_index(name='online_user_coupon_receive')
    X = pd.merge(X, temp, how='left', on='User_id')

    
    X['%online_user_coupon_received'] = X.online_user_coupon_receive / X.online_user_actions

   
    temp = online[(online.Date != date_null) & (online.Coupon_id != 0)]
    temp = temp.groupby('User_id').size().reset_index(name='online_user_coupon_consume')
    X = pd.merge(X, temp, how='left', on='User_id')

   
    X['online_user_couponusage'] = X.online_user_coupon_consume / X.online_user_coupon_receive


    X['%offline_consumedwithcoupon_inO2O'] = X.offline_user_coupon_consume / (X.online_user_coupon_consume + X.offline_user_coupon_consume )

    
    X['%offline_receivedcoupon_inO2O'] = X.offline_user_coupon_receive / (X.online_user_coupon_receive + X.offline_user_coupon_receive)

    print(len(X), len(X.columns))
    print('----------')

    return X

In [ ]:
if __name__ == '__main__':
 
    start = datetime.datetime.now()
    print(start.strftime('%Y-%m-%d %H:%M:%S'))
    date_null = pd.to_datetime('1970-01-01', format='%Y-%m-%d')

    pd.set_option('expand_frame_repr', False)
    pd.set_option('display.max_rows', 200)
    pd.set_option('display.max_columns', 200)

 
    FeaturePath = '/content/drive//MyDrive/Tmall_O2O/'

    off_train, on_train, off_test = get_source_data()

    
    off_train = null_process_offline(off_train, predict=False)
    on_train = null_process_online(on_train)
    off_test = null_process_offline(off_test, predict=True)

    ProcessDataSet1, ProcessDataSet2, ProcessDataSet3 = data_process(off_train, on_train, off_test)

    ProcessDataSet1.to_csv(os.path.join(FeaturePath, 'ProcessDataSet1.csv'), index=None)

    ProcessDataSet2.to_csv(os.path.join(FeaturePath, 'ProcessDataSet2.csv'), index=None)

    ProcessDataSet3.to_csv(os.path.join(FeaturePath, 'ProcessDataSet3.csv'), index=None)

    print((datetime.datetime.now() - start).seconds)

2022-05-01 12:47:04
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754884 entries, 0 to 1754883
Data columns (total 7 columns):
 #   Column         Dtype         
---  ------         -----         
 0   User_id        int64         
 1   Merchant_id    int64         
 2   Coupon_id      float64       
 3   Discount_rate  object        
 4   Distance       float64       
 5   Date_received  datetime64[ns]
 6   Date           datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(2), object(1)
memory usage: 93.7+ MB
None
   User_id  Merchant_id  Coupon_id Discount_rate  Distance Date_received       Date
0  1439408         2632        NaN           NaN       0.0           NaT 2016-02-17
1  1439408         4663    11002.0        150:20       1.0    2016-05-28        NaT
2  1439408         2632     8591.0          20:1       0.0    2016-02-17        NaT
3  1439408         2632     1078.0          20:1       0.0    2016-03-19        NaT
4  1439408         2632     8591.0          20

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


132943 11
132943 34
132943 45
----------
252586 11


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


252586 34
252586 45
----------
113640 9
113640 32
113640 43
----------
65
